# Feature Extraction
## Importing Libraries

In [1]:
# import cv2 as cv
import SimpleITK as sitk

import numpy as np
import os
from skimage.feature.texture import greycomatrix
from skimage.feature.texture import greycoprops
from skimage.measure import shannon_entropy
from radiomics import glrlm, glcm
# import pyfeats
import pandas as pd
import multiprocessing as mlp
import math
import feature_extraction as fe

In [2]:
# from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

## Define Feature Extraction functions

### Read dataset images

In [3]:
def read_images(folder = "dataset/train",
                classes = [
                            "normal",
                            "fatty",
                            "cirrhosis"
                        ]):
    image_names = {}
    images = []
    # Get all image names in folders
    for cls in classes:
        image_names[cls] = os.listdir(f'{folder}/{cls}')

    # read all images to list
    for cls in classes:
        for name in image_names[cls]:
            mask = []
            with open(f'dataset/masks/{name[0:-4]}.txt', 'r') as file:
                data = file.read()
                data = data.strip().split('\n')
                for line in data:
                    x, y = line.split(',')
                    mask.append((int(y),int(x)))
            img = sitk.ReadImage(f'{folder}/{cls}/{name}', sitk.sitkUInt8)
            images.append((name, img,cls,mask))
    return images

### Extract ROIs from image

In [4]:
def extract_roi(img, start , size = (32,32)):
    img = sitk.GetArrayFromImage(img)
    roi = img[start[0]:start[0]+size[0],start[1]:start[1]+size[1]]
    mask = np.zeros(img.shape)
    mask[start[0]:start[0]+size[0],start[1]:start[1]+size[1]] = 1
    return roi, mask

### Extract Features from ROIs

In [5]:
def feature_extraction(img, roi_pos):
    roi_mask_arr = []
    for pos in roi_pos:
        roi_mask_arr.append(extract_roi(img, pos))
    
    # 0 45 90 135 degrees
    angles = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]
    
    da_dict = {
        0: "d1_0",
        1: "d1_45",
        2: "d1_90",
        3: "d1_135",
        
        4: "d2_0",
        5: "d2_45",
        6: "d2_90",
        7: "d2_135",
        
        8: "d3_0",
        9: "d3_45",
        10: "d3_90",
        11: "d3_135",
        
    }
    
    feat_arr = []
    for roi, mask in roi_mask_arr:
        features = {}
        
        glcm_mtx = greycomatrix(roi, distances = [1,2,3], angles = angles, levels = 256)
        con = greycoprops(glcm_mtx, 'contrast').flatten()
        hom = greycoprops(glcm_mtx, 'homogeneity').flatten()
        en = greycoprops(glcm_mtx, 'energy').flatten()
        corr = greycoprops(glcm_mtx, 'correlation').flatten()
        
        for j in range(len(da_dict)):
            features[f'contrast_{da_dict[j]}'] = con[j]
            features[f'homogeneity_{da_dict[j]}'] = hom[j]
            features[f'energy_{da_dict[j]}'] = en[j]
            features[f'correlation_{da_dict[j]}'] = corr[j]
            
        features[f'entropy'] = shannon_entropy(roi)
        
        features[f'mean'] = np.mean(roi)
        features[f'variance'] = np.var(roi)
        
        # pyradiomics
        mask = sitk.GetImageFromArray(mask)
        # GLCM features
        glcmFeatures = glcm.RadiomicsGLCM(img, mask)
        glcmFeatures.enableAllFeatures()
        results = glcmFeatures.execute()
        for col in results.keys():
            features[col] = results[col].item()
        
        # GLRLM features
        glrlmFeatures = glrlm.RadiomicsGLRLM(img, mask)
        glrlmFeatures.enableAllFeatures()
        results = glrlmFeatures.execute()
        
        for col in results.keys():
            features[col] = results[col].item()
        
        feat_arr.append(features)
        
    return feat_arr

### Construct dataframe from ROI features

In [6]:
def build_dataframe(images):
    # dataframe consists of features of 1 ROI per image
    # column name roiNum_feature
    data = pd.DataFrame()

    for img, cls, mask in images:
        feat_arr = feature_extraction(img, roi_pos=mask)
        for row in feat_arr:
            row['target'] = cls
            data = data.append(row,ignore_index=True)
    return data

### Construct dataframe using multiprocessing
### Reduced runtime by 82%

In [7]:
def build_with_mlp(images, n=9): 
    pool = mlp.Pool(n)
    results = pool.map(fe.build_dataframe,np.array_split(images,n))
    return results

## Feature Analysis and Selection

### Extract Features and build dataframe

In [8]:
%%time

# images = read_images('dataset/stretched/train')
# mlp_data = build_with_mlp(images)
# data = pd.DataFrame()
# for frame in mlp_data:
#     data = data.append(frame)

# data.set_index('name', drop=True, inplace=True)

# data.to_csv("dataset/train.csv")

data = pd.read_csv('dataset/train.csv', index_col='name')

data.describe()

Wall time: 78 ms


,contrast_d1_0,contrast_d1_135,contrast_d1_45,contrast_d1_90,contrast_d2_0,contrast_d2_135,contrast_d2_45,contrast_d2_90,contrast_d3_0,contrast_d3_135,...,homogeneity_d2_0,homogeneity_d2_135,homogeneity_d2_45,homogeneity_d2_90,homogeneity_d3_0,homogeneity_d3_135,homogeneity_d3_45,homogeneity_d3_90,mean,variance
count,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,...,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000
mean,139.519073,557.327588,463.135449,467.277785,391.737471,557.327588,463.135449,917.244892,604.282498,934.884076,...,0.101468,0.079234,0.088752,0.061216,0.079006,0.060838,0.064240,0.058601,127.256808,644.009706
std,147.724657,443.369671,361.772842,324.136137,375.169488,443.369671,361.772842,663.401978,523.149730,720.873546,...,0.049200,0.036489,0.043281,0.030509,0.039828,0.030627,0.032205,0.030541,43.200174,518.192499
min,2.794355,18.226847,11.097815,14.074597,8.593750,18.226847,11.097815,22.135417,14.056034,21.187778,...,0.027087,0.028164,0.022842,0.018037,0.026600,0.020656,0.021953,0.018234,5.323242,16.139197
25%,33.826613,210.973985,174.550989,193.190020,107.746875,210.973985,174.550989,365.588542,190.837823,366.636111,...,0.065678,0.055286,0.058626,0.042100,0.051134,0.041772,0.044599,0.039480,99.199219,275.739315
50%,66.564516,378.600416,345.700312,385.714718,211.425000,378.600416,345.700312,713.570833,366.661638,692.543333,...,0.093708,0.072779,0.078808,0.055367,0.071948,0.055350,0.056327,0.051946,128.371094,530.385830
75%,218.776210,851.488554,710.325702,711.252520,634.913021,851.488554,710.325702,1362.885417,986.370690,1388.037778,...,0.123839,0.092408,0.105918,0.071139,0.095446,0.070792,0.075588,0.069197,159.161133,843.983488
max,796.198589,2356.312175,2041.617066,1757.277218,1811.351042,2356.312175,2041.617066,3990.372917,2863.830819,4558.567778,...,0.571401,0.557161,0.601520,0.539195,0.556561,0.490969,0.553843,0.494561,242.736328,4191.302077


In [9]:
%%time

# test_images = read_images("dataset/stretched/test")
# mlp_data = build_with_mlp(test_images)
# test_data = pd.DataFrame()
# for frame in mlp_data:
#     test_data = test_data.append(frame)
    
# test_data.set_index('name', drop=True, inplace=True)

# test_data.to_csv("dataset/test.csv")

test_data = pd.read_csv('dataset/test.csv', index_col='name')

test_data.describe()

Wall time: 73 ms


,contrast_d1_0,contrast_d1_135,contrast_d1_45,contrast_d1_90,contrast_d2_0,contrast_d2_135,contrast_d2_45,contrast_d2_90,contrast_d3_0,contrast_d3_135,...,homogeneity_d2_0,homogeneity_d2_135,homogeneity_d2_45,homogeneity_d2_90,homogeneity_d3_0,homogeneity_d3_135,homogeneity_d3_45,homogeneity_d3_90,mean,variance
count,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,...,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000
mean,98.693760,481.666424,391.710732,405.448278,291.567108,481.666424,391.710732,812.619982,475.429606,828.227639,...,0.107538,0.081329,0.094749,0.064927,0.083671,0.065257,0.066878,0.063261,118.604558,612.059621
std,111.941397,409.512881,367.517034,334.222091,301.621589,409.512881,367.517034,711.742505,444.706417,735.899216,...,0.038212,0.027045,0.037523,0.025917,0.031683,0.026520,0.026213,0.026805,43.781055,519.012346
min,5.564516,25.159209,21.857440,22.250000,16.289583,25.159209,21.857440,35.629167,25.405172,34.073333,...,0.039628,0.031445,0.032693,0.026174,0.027532,0.024179,0.028836,0.022595,30.594727,22.063354
25%,28.148185,189.221904,134.213059,152.076109,90.478125,189.221904,134.213059,287.708854,160.724138,281.397222,...,0.077724,0.062667,0.066959,0.044854,0.060323,0.045083,0.048439,0.042672,86.614502,221.020653
50%,46.623992,306.805411,250.070760,286.308972,144.367708,306.805411,250.070760,511.067187,253.303341,518.418333,...,0.106529,0.079249,0.089093,0.061241,0.081235,0.061081,0.061561,0.059077,120.194824,425.679302
75%,148.455645,754.572841,544.886576,608.550907,472.954427,754.572841,544.886576,1233.457812,803.068696,1244.031667,...,0.130661,0.095316,0.114242,0.080179,0.103041,0.078092,0.080720,0.079175,147.738525,810.817033
max,533.570565,1741.885536,2010.729448,1515.122984,1349.643750,1741.885536,2010.729448,3065.783333,1961.033405,3096.735556,...,0.263151,0.222067,0.256477,0.188469,0.223884,0.193968,0.196544,0.174825,235.129883,2777.443015


## Testing

In [10]:
def split(data, test_data, drop):
    X_train = data.copy()#.drop(['name'], axis=1)
    y_train = X_train.pop('target')
    X_test = test_data.copy()#.drop(['name'], axis=1)
    y_test = X_test.pop('target')

    X_train = X_train[y_train != drop]
    X_test = X_test[y_test != drop]

    y_train = y_train[y_train != drop]
    y_test = y_test[y_test != drop]

    std = StandardScaler()
    std.fit(X_train)
    X_train = pd.DataFrame(std.transform(X_train), columns = X_train.columns, index = X_train.index)
    X_test = pd.DataFrame(std.transform(X_test), columns = X_test.columns, index = X_test.index)
    return X_train, y_train, X_test, y_test, std

In [11]:
def train_test(model, X_train, y_train, X_test, y_test):
    model = model.fit(X_train, y_train)
    y_pred = pd.Series(model.predict(X_test),index=y_test.index)
    return model, y_pred

In [12]:
def images_pred(y_pred):
    count = 0
    prediction = {}
    for name in np.unique(y_pred.index):
        pred_cls = {}
        for i in y_pred[name]:
            if i not in pred_cls.keys():
                pred_cls[i]=1
            else: pred_cls[i]+=1
        
        prediction[name] = max(pred_cls, key=pred_cls.get)
    return prediction

In [13]:
def images_acc(y_test, y_pred):
    pred_count = 0
    for key in y_pred.keys():
        if y_test[key][0] == y_pred[key]:
            pred_count += 1
    return pred_count/len(y_pred.keys())

In [14]:
models = {
    "RFC": RandomForestClassifier(
                    random_state=42,
                    max_features='auto',
                    n_estimators= 500,
                    max_depth=6,
                    criterion='entropy'),
    "MLP": MLPClassifier(
                    max_iter=600,
                    momentum=0.6,
                    solver='adam',
                    activation='relu',
                    learning_rate_init=0.005,
                    alpha=0.001),
    "SVC": svm.SVC()
}
classes = ['normal', 'fatty', 'cirrhosis']

for drop in classes:
    X_train, y_train, X_test, y_test, std = split(data, test_data, drop)
    print(*[cls for cls in classes if cls != drop])
    for name in models.keys():
        model, y_pred = train_test(models[name], X_train, y_train, X_test, y_test)
        prediction = images_pred(y_pred)
        print(name," Image Accuracy: ", images_acc(y_test, prediction))
        report = classification_report(y_test, y_pred, output_dict = True)
        cr = pd.DataFrame(report).transpose()
        print(cr)
    print('\n\n')

fatty cirrhosis
RFC  Image Accuracy:  0.8571428571428571
              precision    recall  f1-score     support
cirrhosis      0.666667  0.633333  0.649573   60.000000
fatty          0.770833  0.795699  0.783069   93.000000
accuracy       0.732026  0.732026  0.732026    0.732026
macro avg      0.718750  0.714516  0.716321  153.000000
weighted avg   0.729984  0.732026  0.730717  153.000000
MLP  Image Accuracy:  0.8571428571428571
              precision    recall  f1-score     support
cirrhosis      0.736842  0.700000  0.717949   60.000000
fatty          0.812500  0.838710  0.825397   93.000000
accuracy       0.784314  0.784314  0.784314    0.784314
macro avg      0.774671  0.769355  0.771673  153.000000
weighted avg   0.782830  0.784314  0.783260  153.000000
SVC  Image Accuracy:  0.8571428571428571
              precision    recall  f1-score     support
cirrhosis      0.687500  0.733333  0.709677   60.000000
fatty          0.820225  0.784946  0.802198   93.000000
accuracy       0.7647

In [18]:
normal_fatty_mlp = MLPClassifier(
                    max_iter=600,
                    momentum=0.6,
                    solver='adam',
                    activation='relu',
                    learning_rate_init=0.005,
                    alpha=0.001)
X_train, y_train, X_test, y_test, normal_fatty_std = split(data, test_data, 'cirrhosis')
model, y_pred = train_test(normal_fatty_mlp, X_train, y_train, X_test, y_test)
normal_fatty_mlp = model
prediction = images_pred(y_pred)
print("Normal/Fatty MLP Image Accuracy: ", images_acc(y_test, prediction))
report = classification_report(y_test, y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)

Normal/Fatty MLP Image Accuracy:  0.8125
              precision    recall  f1-score     support
fatty          0.732673  0.795699  0.762887   93.000000
normal         0.698413  0.619718  0.656716   71.000000
accuracy       0.719512  0.719512  0.719512    0.719512
macro avg      0.715543  0.707709  0.709802  164.000000
weighted avg   0.717841  0.719512  0.716923  164.000000


In [16]:
normal_cirrhosis_mlp = MLPClassifier(
                    max_iter=600,
                    momentum=0.6,
                    solver='adam',
                    activation='relu',
                    learning_rate_init=0.005,
                    alpha=0.001)
X_train, y_train, X_test, y_test, normal_cirrhosis_std = split(data, test_data, 'fatty')
model, y_pred = train_test(normal_cirrhosis_mlp, X_train, y_train, X_test, y_test)
normal_cirrhosis_mlp = model
prediction = images_pred(y_pred)
print("normal/cirrhosis MLP Image Accuracy: ", images_acc(y_test, prediction))
report = classification_report(y_test, y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)

normal/cirrhosis MLP Image Accuracy:  0.7
              precision    recall  f1-score     support
cirrhosis      0.500000  0.466667  0.482759   60.000000
normal         0.573333  0.605634  0.589041   71.000000
accuracy       0.541985  0.541985  0.541985    0.541985
macro avg      0.536667  0.536150  0.535900  131.000000
weighted avg   0.539746  0.541985  0.540362  131.000000


In [24]:
fatty_cirrhosis_mlp = MLPClassifier(
                    max_iter=600,
                    momentum=0.6,
                    solver='adam',
                    activation='relu',
                    learning_rate_init=0.005,
                    alpha=0.001)
X_train, y_train, X_test, y_test, fatty_cirrhosis_std = split(data, test_data, 'normal')
model, y_pred = train_test(fatty_cirrhosis_mlp, X_train, y_train, X_test, y_test)
fatty_cirrhosis_mlp = model
prediction = images_pred(y_pred)
print("cirrhosis/Fatty MLP Image Accuracy: ", images_acc(y_test, prediction))
report = classification_report(y_test, y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)

cirrhosis/Fatty MLP Image Accuracy:  1.0
              precision    recall  f1-score     support
cirrhosis      0.765625  0.816667  0.790323   60.000000
fatty          0.876404  0.838710  0.857143   93.000000
accuracy       0.830065  0.830065  0.830065    0.830065
macro avg      0.821015  0.827688  0.823733  153.000000
weighted avg   0.832962  0.830065  0.830939  153.000000


In [25]:
models = {
    "normal_fatty": (normal_fatty_mlp, normal_fatty_std),
    "normal_cirrhosis": (normal_cirrhosis_mlp, normal_cirrhosis_std),
    "fatty_cirrhosis": (fatty_cirrhosis_mlp, fatty_cirrhosis_std)
}

X_test = test_data.copy()
y_test = X_test.pop('target')

std = StandardScaler()
std.fit(X_train)
X_test = pd.DataFrame(std.transform(X_test), columns = X_test.columns, index = X_test.index)

In [26]:
pd.set_option('display.max_rows', None)
predictions = {}
for name in models.keys():
    X_test = test_data.copy()
    y_test = X_test.pop('target')
    X_test = pd.DataFrame(models[name][1].transform(X_test), columns = X_test.columns, index = X_test.index)
    y_pred = pd.Series(models[name][0].predict(X_test),index=y_test.index)
    predictions[name] = images_pred(y_pred)
    
image_names = np.unique(y_test.index)

In [27]:
image_name = np.unique(y_test.index)
final_pred = {}
for image in image_name:
    pred = {
        'normal': 0,
        'fatty': 0,
        'cirrhosis': 0
    }
    for model in predictions.keys():
        pred[predictions[model][image]] += 1
    cls = max(pred, key=pred.get)
    if pred[cls] == 1:
        final_pred[image] = 'abstain'
    else: final_pred[image] = cls

In [28]:
images_acc(y_test, final_pred)

0.75

In [30]:
y_test = y_test[~y_test.index.duplicated(keep='first')].sort_index()
y_pred = pd.Series(final_pred).sort_index()
abstain = y_pred[y_pred=='abstain'].index

y_test = y_test.drop(abstain)
y_pred = y_pred.drop(abstain)

report = classification_report(y_test, y_pred, output_dict = True)
cr = pd.DataFrame(report).transpose()
print(cr)
print("Abstension Rate: ", len(abstain)/(len(y_pred)+len(abstain)))

              precision    recall  f1-score    support
cirrhosis      0.666667  0.500000  0.571429   4.000000
fatty          0.900000  1.000000  0.947368   9.000000
normal         0.666667  0.666667  0.666667   6.000000
accuracy       0.789474  0.789474  0.789474   0.789474
macro avg      0.744444  0.722222  0.728488  19.000000
weighted avg   0.777193  0.789474  0.779581  19.000000
Abstension Rate:  0.05


In [31]:
from joblib import dump

for name in models.keys():
    dump(models[name][0], f'dataset/models/{name}_mlp.joblib') 
    dump(models[name][1], f'dataset/models/{name}_std.joblib') 